In [ ]:
# Colab setup\n!nvidia-smi\n!pip -q install -U ultralytics pycocotools matplotlib tqdm opencv-python git+https://github.com/ifzhang/ByteTrack.git labellerr-sdk\nfrom ultralytics import YOLO\nimport os, json, glob, cv2, numpy as np\n

In [ ]:
# If your repo is in Drive, mount and cd there (edit the path to your folder name)\nfrom google.colab import drive\ndrive.mount('/content/drive')\nos.chdir('/content/drive/MyDrive/first_lastname')  # <-- change if needed\nprint('CWD:', os.getcwd())

In [ ]:
# Train YOLOv8-seg on your dataset (expects data/coco.yaml)\nmodel = YOLO('yolov8n-seg.pt')\nresults = model.train(data='data/coco.yaml', epochs=50, imgsz=640, batch=16, name='pf_seg', device=0, seed=42)\nprint(results)

In [ ]:
# Inference on test set and save predictions + visualizations\nbest_w = 'runs/segment/pf_seg/weights/best.pt'\nassert os.path.exists(best_w), 'best.pt not found; ensure training finished'\nmodel = YOLO(best_w)\ntest_imgs = sorted(glob.glob('data/yolo_data/images/test/*'))\nos.makedirs('predictions/images', exist_ok=True)\npred_meta = []\nfor ip in test_imgs:\n    r = model.predict(ip, save=False, imgsz=640, conf=0.25)[0]\n    dets = []\n    boxes = r.boxes\n    masks = r.masks.data.cpu().numpy() if r.masks is not None else []\n    for i in range(len(boxes)):\n        xyxy = boxes.xyxy[i].tolist()\n        cls = int(boxes.cls[i].item())\n        score = float(boxes.conf[i].item())\n        # save binary mask as RLE-like list (simple fallback): indices of foreground\n        mask_points = None\n        if len(masks):\n            m = (masks[i] > 0.5).astype(np.uint8)\n            ys, xs = np.where(m==1)\n            mask_points = list(map(lambda ab: [int(ab[1]), int(ab[0])], zip(xs, ys)))[:2000]  # cap for size\n        dets.append({\"bbox_xyxy\": xyxy, \"cls\": cls, \"score\": score, \"mask_points\": mask_points})\n    img_plot = r.plot()[:, :, ::-1]\n    outp = os.path.join('predictions/images', os.path.basename(ip))\n    cv2.imwrite(outp, img_plot)\n    pred_meta.append({\"image\": os.path.basename(ip), \"detections\": dets})\nwith open('predictions/results.json','w') as f: json.dump(pred_meta, f, indent=2)\nlen(pred_meta)